# Centrifugation Simulation — RCF ⇄ RPM & Sedimentation


In [ ]:
%matplotlib inline
import numpy as np, matplotlib.pyplot as plt
import ipywidgets as W
from IPython.display import display, clear_output

def rcf_to_rpm(rcf, radius_cm): return np.sqrt(rcf/(1.118e-5*radius_cm))*60
def rpm_to_rcf(rpm, radius_cm): return 1.118e-5*radius_cm*(rpm/60)**2

def tube_profile(particle_density, medium_density, n=200):
    x = np.linspace(0,1,n)
    grad = medium_density + 0.2*(x-0.5)
    drift = (particle_density - grad)*2
    conc = np.clip(0.5 + np.cumsum(drift)/n, 0, 1)
    return x, conc

radius = W.FloatSlider(value=10, min=5, max=20, step=0.5, description='Radius (cm)')
rpm = W.FloatSlider(value=12000, min=500, max=20000, step=100, description='RPM')
rcf = W.FloatSlider(value=rpm_to_rcf(12000,10), min=100, max=30000, step=50, description='RCF (g)')
lock = W.ToggleButtons(options=['Control RPM','Control RCF'], value='Control RPM', description='Mode')
rho_p = W.FloatSlider(value=1.25, min=1.0, max=1.6, step=0.01, description='ρ particle')
rho_m = W.FloatSlider(value=1.1, min=1.0, max=1.4, step=0.01, description='ρ medium')

out = W.Output()

def sync(*_):
    if lock.value=='Control RPM':
        rcf.value = rpm_to_rcf(rpm.value, radius.value)
    else:
        rpm.value = rcf_to_rpm(rcf.value, radius.value)

for w in [radius, rpm, rcf, lock]: w.observe(sync, 'value')

def update(*_):
    with out:
        clear_output(wait=True)
        x, c = tube_profile(rho_p.value, rho_m.value)
        plt.figure(figsize=(6,4)); plt.plot(x, c, lw=3)
        plt.xlabel('Tube length (relative)'); plt.ylabel('Particle concentration (arb)')
        plt.title(f'RPM={rpm.value:.0f}, RCF={rcf.value:.0f} g | ρp={rho_p.value:.2f}, ρm={rho_m.value:.2f}')
        plt.grid(True); plt.show()

for w in [radius, rpm, rcf, lock, rho_p, rho_m]: w.observe(update, 'value')
display(W.VBox([W.HBox([radius, lock]), W.HBox([rpm, rcf]), W.HBox([rho_p, rho_m]), out]))
sync(); update()
